## LangGraph Basics

> https://www.youtube.com/watch?v=ny215UUXbhI

---

In [18]:
GROQ_API_KEY = ""

In [19]:
from langchain_groq import ChatGroq
from langgraph.graph import Graph

In [20]:
def setup_model():
    
    llm = llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=GROQ_API_KEY
    # other params...
    )

    return llm

In [21]:
llm = setup_model()

In [22]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='I love programming in French is: "J\'aime programmer."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 30, 'total_tokens': 46, 'completion_time': 0.024341526, 'prompt_time': 0.002687649, 'queue_time': 0.01042447, 'total_time': 0.027029175}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-2b6e115d-8baf-40c4-a2d6-ff51895a491a-0', usage_metadata={'input_tokens': 30, 'output_tokens': 16, 'total_tokens': 46})

### Simple Graphs

In [23]:
def function1(input: str) -> str:
    return input + " First Function!"

def function2(input: str) -> str:
    return input + " Second Function!"

In [24]:
workflow = Graph()

workflow.add_node("node_1", function1)
workflow.add_node("node_2", function2)

In [25]:
workflow.add_edge(start_key="node_1", end_key="node_2")

In [26]:
workflow.set_entry_point(key="node_1")
workflow.set_finish_point(key="node_2")

In [27]:
app = workflow.compile()

In [28]:
app.invoke("hello from")

'hello from First Function! Second Function!'

In [30]:
input = "hello from"

for output in app.stream(input=input):
    print (output)

{'node_1': 'hello from First Function!'}
{'node_2': 'hello from First Function! Second Function!'}


### Graph with simple LLM call

In [32]:
def function1(input: str) -> str:
    prompt_message = f"""
Your job is to generate the user intent among the following : [general-query, troubleshooting, unknown]. 
Only output a single word form the given list, nothing else.
Here is the user query:
{input}
"""
    response = llm.invoke(prompt_message)
    return response.content

def function2(input: str) -> str:
    intent_responses = {
        "general-query": "RAG",
        "troubleshooting": "playbooks",
        "unknown": "follow-up"
    }

    action = intent_responses.get(input, "follow-up")

    return f"The action for intent: {input} is {action}"


In [33]:
workflow = Graph()

workflow.add_node("Agent", function1)
workflow.add_node("Tool", function2)
workflow.add_edge(start_key="Agent", end_key="Tool")

workflow.set_entry_point(key="Agent")
workflow.set_finish_point(key="Tool")

app = workflow.compile()

In [34]:
query = "Hey! How are you doing?"
app.invoke(query)

'The action for intent: general-query is RAG'

In [35]:
query = "My laptop is so slow"
app.invoke(query)

'The action for intent: troubleshooting is playbooks'

In [36]:
input = "My laptop is so slow"

for output in app.stream(input=input):
    print (output)

{'Agent': 'troubleshooting'}
{'Tool': 'The action for intent: troubleshooting is playbooks'}


### RAG Pipeline Integration